# SETUP, DATASET
- - -

In [1]:
import numpy as np
import pandas as pd
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer

from konlpy.tag import *
from soynlp.word import WordExtractor

import re

hannanum = Hannanum()
kkma = Kkma()
twitter = Okt()
komoran = Komoran()

In [17]:
cs_df = pd.read_pickle('cs_whole.pkl')
cs_df.tail()

,date,title,body
1017,2017.03.06,"[사설] 운명의 일주일, '탄핵' '기각' 이후가 더 중요하다",이르면 이번 주 후반 박근혜 대통령 탄핵 심판 사건에 대한 헌재 결정이 나올 가능성...
1018,2017.03.07,[사설] 중국 땅인지 착각게 한 롯데 앞 촛불 시위대,지난 4일 밤 광주 롯데백화점 앞 촛불 시위 사진은 눈을 의심케 했다. 수백 명이 ...
1019,2017.03.07,"[사설] 특검, '최순실 국정농단'의 本流 꿰뚫은 수사였나",박영수 특검팀이 6일 수사 결과를 발표했다. 그러나 평가는 엇갈린다. 구속기소된 이...
1020,2017.03.09,"[사설] 10일 탄핵 심판 선고, 모두 自重하고 또 自制하자",박근혜 대통령 탄핵 심판 선고 기일이 10일로 확정됐다. 헌재 재판관 8명이 8일 ...
1021,2017.03.10,"[사설] 오늘 시험대 오르는 대한민국, '역사적 승복'으로 위기 끝내자",헌법재판소가 오늘 11시 박근혜 대통령 탄핵심판 사건을 선고한다. 작년 10월 5일...


In [18]:
sentiment_df = pd.read_csv('/Users/quartz/Dropbox/Heart-Signal-BlueHouse/lexicon/polarity.csv')
sentiment_df.tail()

,ngram,freq,COMP,NEG,NEUT,None,POS,max.value,max.prop
16357,힘들/VA;ㄹ/ETM;것/NNB,1,0.0,1.0,0.0,0.0,0.0,NEG,1.0
16358,힘들/VA;ㄹ/ETM;때/NNG,1,0.0,1.0,0.0,0.0,0.0,NEG,1.0
16359,힘차/VA,1,0.0,1.0,0.0,0.0,0.0,NEG,1.0
16360,힘차/VA;ㄴ/ETM,1,0.0,1.0,0.0,0.0,0.0,NEG,1.0
16361,힘차/VA;ㄴ/ETM;붓/NNG,1,0.0,1.0,0.0,0.0,0.0,NEG,1.0


In [66]:
sentiment_df[sentiment_df.ngram.str.contains('재미')]

,ngram,freq,COMP,NEG,NEUT,None,POS,max.value,max.prop
3969,다른/MM;재미/NNG,1,0.0,0.0,0.0,0.0,1.0,POS,1.0
3970,다른/MM;재미/NNG;로/JKB,1,0.0,0.0,0.0,0.0,1.0,POS,1.0
4286,더욱/MAG;재미있/VA,1,0.0,0.0,0.0,0.0,1.0,POS,1.0
5049,또/MAG;다른/MM;재미/NNG,1,0.0,0.0,0.0,0.0,1.0,POS,1.0
13246,재미/NNG,5,0.0,0.0,0.0,0.0,1.0,POS,1.0
13247,재미/NNG;가/JKS,2,0.0,0.0,0.0,0.0,1.0,POS,1.0
13248,재미/NNG;가/JKS;쏠쏠/XR,1,0.0,0.0,0.0,0.0,1.0,POS,1.0
13249,재미/NNG;가/JKS;있/VV,1,0.0,0.0,0.0,0.0,1.0,POS,1.0
13250,재미/NNG;로/JKB,1,0.0,0.0,0.0,0.0,1.0,POS,1.0
13251,재미/NNG;로/JKB;눈길/NNG,1,0.0,0.0,0.0,0.0,1.0,POS,1.0


In [45]:
sentiment_columns = sentiment_df.columns

for column in sentiment_columns:

    frequent = sentiment_df[column].value_counts().iloc[:10]
    print("column : {} \n\n --list-- \n {} \n\n".format(column, frequent))

column : ngram 

 --list-- 
 다/JX                 1
이한탁/NNP              1
예술의전당/NNP            1
지역/NNG;호텔/NNG        1
입/VV                 1
못/MAG;내놓/VV          1
을/JKO;늦추/VV;지/EC     1
을/JKO;입히/VV          1
새끼/NNG;젖/NNG;도/JX    1
고/JKQ;말/NNG;하/XSV    1
Name: ngram, dtype: int64 


column : freq 

 --list-- 
 1     13633
2      1404
3       486
4       226
5       142
6       113
7        49
8        41
9        35
11       24
Name: freq, dtype: int64 


column : COMP 

 --list-- 
 0.000000    15769
1.000000      370
0.500000       52
0.333333       24
0.200000       18
0.142857       11
0.166667       11
0.250000       10
0.058824        5
0.111111        5
Name: COMP, dtype: int64 


column : NEG 

 --list-- 
 0.000000    8899
1.000000    6124
0.500000     554
0.333333     172
0.666667      93
0.250000      72
0.400000      46
0.200000      41
0.750000      36
0.166667      27
Name: NEG, dtype: int64 


column : NEUT 

 --list-- 
 0.000000    14901
1.000000      900
0.500000    

# Practice
- - -

### 2012년 12월 20일 (중앙일보) 테스트

In [19]:
corpus = cs_df.body[0]
corpus = re.sub('\n', '', corpus)
corpus[:500]

'19일 실시된 대통령선거에서 새누리당 박근혜 후보가 투표자 3072만명 중 51% 남짓 1600만표가량을 얻어 18대 대통령에 당선됐다. 민주통합당 문재인 후보는 48% 남짓 1500만표가량을 얻었다. 1987년 직선제 개헌 이후 득표율 50%를 넘긴 대통령은 이번이 처음이다. 70%를 약간 넘을 것으로 예상했던 투표율은 75.8%까지 올랐다. 대한민국 건국 이후 첫 여성 대통령이 탄생했다. 박 후보의 당선으로 투표율이 높으면 야당이 유리하다는 속설(俗說)도 깨졌다.경쟁자 지지 국민의 박탈감 헤아려야정권 재창출보다 정권 교체를 바라는 여론이 더 높은 가운데 치러진 이번 대선은 박 후보에게 버거운 선거였다. 선거 초반 한동안 유지되던 박근혜 대세론은 무소속 안철수 후보 등장으로 몇 차례 크게 흔들렸다. 박 후보는 박근혜·문재인·안철수 3자 가상(假想) 대결에선 늘 1위를 지켰으나 야당 단일 후보에겐 밀리는 결과가 몇 번이나 나타났다. 안 후보 사퇴 이후 박 후보에게 10%포인트 이상 '

In [46]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(ngram_range=(1, 2))
X_sparse = count_vect.fit_transform([corpus])

In [47]:
_word2int = count_vect.vocabulary_
_int2word = [word for word, _ in sorted(_word2int.items(), key=lambda x:x[1])]

def word2int(word):
    return _word2int.get(word, -1)

def int2word(idx):
    if 0 <= idx < len(_int2word):
        return _int2word(idx)
    return None

In [48]:
word2int('박근혜')

283

In [49]:
# make sentence

sentences = kkma.sentences(corpus)
sentences[:10]

['19일 실시된 대통령 선거에서 새 누리 당 박근 혜 후보가 투표자 3072만명 중 51% 남짓 1600만 표 가량을 얻어 18대 대통령에 당선됐다.',
 '민주통합당 문 재인 후보는 48% 남짓 1500만 표 가량을 얻었다.',
 '1987년 직선제 개헌 이후 득표율 50%를 넘긴 대통령은 이번이 처음이다.',
 '70%를 약간 넘을 것으로 예상했던 투표율은 75.8%까지 올랐다.',
 '대한민국 건국 이후 첫 여성 대통령이 탄생했다.',
 '박 후보의 당선으로 투표율이 높으면 야당이 유리 하다는 속설( 俗說) 도 깨졌다.',
 '경쟁자 지지 국민의 박탈감 헤아려야 정권 재창출보다 정권 교체를 바라는 여론이 더 높은 가운데 치러 진 이번 대선은 박 후보에게 버거운 선거였다.',
 '선거 초반 한동안 유지되던 박근 혜 대세론은 무소속 안 철수 후보 등장으로 몇 차례 크게 흔들렸다.',
 '박 후보는 박근 혜· 문 재인· 안 철수 3 자 가상( 假想) 대결에선 늘 1위를 지켰으나 야당 단일 후보에 겐 밀리는 결과가 몇 번이나 나타났다.',
 '안 후보 사퇴 이후 박 후보에게 10% 포인트 이상 뒤지 던 문 후보의 지지율이 꾸준히 올라가 선거 이틀 전에는 뒤집히기도 했다.']

In [54]:
document_score = []

for sentence in sentences:

    # scores, vocabulary
    sentence_score = []
    vocab = sentiment_df.ngram.str
    
    # count_vect (text to vect)
    count_vect.fit([sentence])
    count_keys = count_vect.vocabulary_.keys()
    words = list(count_keys)

    # sentiment analysis for word
    for word in words:
        
        sub_df = sentiment_df[vocab.contains(word, regex=False)]
        if len(sub_df) == 0:
            continue
        else:
            score = np.mean(-sub_df.NEG + sub_df.POS)
            sentence_score.append(score)
    
    result = sum(sentence_score)
    document_score.append(result)
    
    print("{} | score = {}".format(sentence, result))
    
sum(document_score)

14.520678984401687

### 2012년 12월 20일(~30일) vs. 2017년 3월 1일(~10일)

In [56]:
cs_df.tail()

,date,title,body
1017,2017.03.06,"[사설] 운명의 일주일, '탄핵' '기각' 이후가 더 중요하다",이르면 이번 주 후반 박근혜 대통령 탄핵 심판 사건에 대한 헌재 결정이 나올 가능성...
1018,2017.03.07,[사설] 중국 땅인지 착각게 한 롯데 앞 촛불 시위대,지난 4일 밤 광주 롯데백화점 앞 촛불 시위 사진은 눈을 의심케 했다. 수백 명이 ...
1019,2017.03.07,"[사설] 특검, '최순실 국정농단'의 本流 꿰뚫은 수사였나",박영수 특검팀이 6일 수사 결과를 발표했다. 그러나 평가는 엇갈린다. 구속기소된 이...
1020,2017.03.09,"[사설] 10일 탄핵 심판 선고, 모두 自重하고 또 自制하자",박근혜 대통령 탄핵 심판 선고 기일이 10일로 확정됐다. 헌재 재판관 8명이 8일 ...
1021,2017.03.10,"[사설] 오늘 시험대 오르는 대한민국, '역사적 승복'으로 위기 끝내자",헌법재판소가 오늘 11시 박근혜 대통령 탄핵심판 사건을 선고한다. 작년 10월 5일...


In [59]:
cs_df.date[0]

'2012.12.20'

In [57]:
# stop_words = ['가/JKS',
#  '으로/JKM',
#  '되/XSV',
#  '었/EPT',
#  '다/EFN',
#  '은/JX',
#  '도/JX',
#  '화려/XR',
#  '하/XSA',
#  'ㄴ/ETD',
#  '아/ECS',
#  '은/ETD',
#  '에서/JKM',
#  '는/JX',
#  '지만/ECE',
#  '보다/JKM',
#  '아/ECD',
#  '있/VXV',
#  '는/ETD',
#  '이/JKS',
#  '하/XSV',
#  '은/JKS',
#  '의/JKG',
#  '에/JKM',
#  '을/JKO',
#  'ㄹ/ETD',
#  '란/ETD',
#  '나/JC',
#  '와/JC',
#  '를/JKO',
#  '로/JKM',
#  '았/EPT',
#  '고/ECE',
#  '으며/ECE',
#  '이/JKC',
#  '라는/ETD',
#  '지/ECD',
#  '않/VXV',
#  '는/EPT',
#  '아야/ECD',
#  'ㄴ다/EFN',
#  '과/JKM',
#  'ㄴ데/ECE',
#  '미약/XR',
#  '위태/XR',
#  '롭/XSA',
#  '정/XSN',
#  '면서/ECE',
#  '며/ECE',
#  '라도/JX',
#  '면/ECE',
#  '에게/JKM',
#  '가/JKC',
#  '과/JC',
#  '어야/ECD',
#  '어/ECS',
#  '려는/ETD',
#  'ㄴ단/ETD',
#  '극심/XR',
#  '처럼/JKM',
#  '부터/JX',
#  '을/ETD',
#  '여차/XR',
#  '려면/ECE',
#  '와/JKM',
#  '다/ECS',
#  '과감/XR',
#  '겠/EPT',
#  '다는/ETD',
#  '게/ECD',
#  '뿐/JX',
#  '라/ECD',
#  'ㄴ가/EFQ',
#  '막대/XR',
#  '나/ECE',
#  '성/XSN',
#  '내/VXV',
#  '들/XSN',
#  '어서/ECD',
#  '어/ECD']

In [29]:
sentences

['어제 헌법재판소가 8 인 재판관 만장일치로 박근 혜 대통령에게 파면을 결정· 선고했다.',
 '박 전 대통령은 자연인 신분으로 돌아갔다.',
 '말 그대로 ‘ 폐위’ 된 것이다.',
 '현직 대통령이 탄핵으로 파면 된 것은 헌정 사상 처음이다.',
 '안타까운 국가적 비극 임과 동시에 대한민국 최악의 흑역사 중 하나로 남게 됐다.',
 '하지만 결론부터 말하면 우리는 헌재의 판결을 국민의 명령으로 받아들이며, 이를 지지한다.',
 '이 판결에 모두 승복해 법치의 새 역사를 열어야 한다고 본다.',
 '8 인 재판관, 만장일치 파면 최 순실 사익추구 지원· 개입 박 대통령, 승복 선언부터 해야 이날 헌재는 박 대통령에 대한 다섯 가지 탄핵 사유 중 두 가지를 직접적 탄핵 인용 근거로 판단했다.',
 '최 순 실의 국정 농단을 허용· 방 조하고 그녀의 사익 추구를 지원하기 위해 대통령 권한을 남용, 헌법과 법률을 위반했다는 것이다.',
 '헌재는 구체적으로 대기업 출연금으로 만든 미르 ·K 스포츠재단의 설립· 운영· 의사결정에 관여했으며 KD 코퍼 레이션· 플레이 그라운드· 더블루 K 등을 통한 이권 추구 과정을 지원했다고',
 '적시했다.',
 '이정 미 헌재 소장 권한 대행은 결정문에서 “ 대통령의 이런 행위는 헌법· 국가공무원 법· 공직자 윤리법 등을 위배해 대의 민주주의 제와 법치주의 정신을 훼손하는 것” 이라고 밝혔다.',
 '또 “ 대통령은 최씨의 국정 개입 사실을 철저히 숨기거나 부인하고 오히려 의혹 제기를 비난해 국회의 견제와 언론의 감시장치가 제대로 작동될 수 없었다” 고 지적했다.',
 '국정은 비선 실세가 아닌 공조직에 맡겨 공정하고 투명하게 운영한 뒤 국민의 평가를 받아야 한다는 것이다.',
 '헌재가 결정문에서 “ 대통령이 최씨에게 도움을 주는 과정에서 기업의 재산권을 침해하고 기업 경영의 자유를 침해했다” 고 밝힌 대목은 주목할 필요가 있다.',
 '두 재단에 기금을 출연한 대기업들을 뇌물죄로 사법처리하려는 특검과 달리 강요의 피해자로

In [ ]:
sentences = kkma.sentences(ja_df.body[0])
scores = []

tv = TfidfVectorizer(ngram_range=(1, 1), tokenizer=tokenize_pos)

for sentence in sentences:
    sub = []
    sub.append(sentence)
    tv.fit(sub)
    words = list(tv.vocabulary_.keys())

    for word in words:
        df = sentiment_df[sentiment_df.ngram.str.contains(word, regex=False)]
        if len(df) == 0:
            continue
        else:
            score = np.mean(-df.NEG + df.POS)
            scores.append(score)
    
    result = sum(scores)
    print("{} | score = {}".format(sentence, result))

In [23]:
for i in range(10):
    date = ja_df.date[i]
    title = ja_df.title[i]
    sentences= kkma.sentences(ja_df.body[i])
    
    scores = []

    tv = TfidfVectorizer(ngram_range=(1, 1), tokenizer=tokenize_pos)
    tv.fit(sentences)

    words = list(tv.vocabulary_.keys())

    for word in words:
        df = sentiment_df[sentiment_df.ngram.str.contains(word, regex=False)]
        if len(df) == 0:
            continue
        else:
#             print(word)
            score = np.mean(-df.NEG + df.POS)
            scores.append(score)
            
    result = sum(scores)

    print("{} | {} | score = {}".format(date, title, result))

2017.03.10 | [사설] 헌재의 대통령 파면은 국민의 명령이다 | score = -5.141954564332643
2017.03.10 | [사설] 대선후보, 대통령 파면에서 무엇을 새겨야 하나 | score = 10.448805746480343
2017.03.10 | [사설] 오늘 승복이 법치의 새 역사 연다 | score = 10.614208728691608
2017.03.09 | [사설] 정치가 복원돼야 갈등 치유가 가능해진다 | score = -19.66702122373558
2017.03.08 | [사설] 3월 10일, 분열과 갈등의 끝이어야 한다 | score = -6.807482638058782
2017.03.06 | [사설] 특검은 재판, 검찰은 우병우 재수사에 매진하라 | score = -4.747493260028758
2017.03.05 | [사설] 운명의 한 주… 탄핵 이후를 준비할 때 | score = -6.116814258512559
2017.03.02 | [사설] 집회 전날 지지층 결집 부추긴 박 대통령 | score = 5.175375392298785
2017.03.02 | [사설] ‘대한독립 만세’ 98주년 … 갈라진 민심, 쪼개진 광장 | score = -12.046906760792451
2017.02.28 | [사설] 탄핵 찬반 선동이 3·1절 정신과 무슨 관계 있나 | score = 19.799383052835125


In [53]:
for i in range(10):
    i += 1211
    date = ja_df.date[i]
    title = ja_df.title[i]
    sentences= kkma.sentences(ja_df.body[i])
    
    scores = []

    tv = TfidfVectorizer(ngram_range=(1, 1), tokenizer=tokenize_pos)
    tv.fit(sentences)

    words = list(tv.vocabulary_.keys())

    for word in words:
        df = sentiment_df[sentiment_df.ngram.str.contains(word, regex=False)]
        if len(df) == 0:
            continue
        else:
#             print(word)
            score = np.mean(-df.NEG + df.POS)
            scores.append(score)
            
    result = sum(scores)

    print("{} | {} | score = {}".format(date, title, result))

2012.12.27 | [사설] 상생과 성장의 두 마리 토끼를 잡으려면 | score = 6.378465291644664
2012.12.27 | [사설] 피와 땀과 눈물의 대한민국역사박물관 | score = 2.710766561879451
2012.12.26 | [사설] ‘국민 대통합’ 인사 원칙 지켜져야 | score = 9.952310887285478
2012.12.25 | [사설] 지하경제 양성화만으론 복지 공약 못 지킨다 | score = 4.205365751524482
2012.12.25 | [사설] 제2 ‘고소영’‘강부자’ 파동 없도록 | score = 5.253365400615667
2012.12.24 | [사설] 박근혜·시진핑·아베, 과거 아닌 미래를 보라 | score = 24.954652406975935
2012.12.22 | [사설] 내년 예산안을 누더기로 만들 것인가 | score = 15.64285912625559
2012.12.22 | [사설] 제2, 제3의 이학재 나와야 한다 | score = 12.809316761186546
2012.12.21 | [사설] 취임까지 67일이 5년 성패 좌우한다 | score = 17.474580334934757
2012.12.20 | [사설] 여성 대통령 박근혜 … 화려한 기록, 무거운 짐 | score = 3.6937356137825272


In [199]:
for i in range(50):
    date = ja_df.date[i]
    title = ja_df.title[i]
    sentences= kkma.sentences(ja_df.body[i])
    
    scores = []

    tv = TfidfVectorizer(ngram_range=(1, 1), tokenizer=tokenize_pos, stop_words=stop_words)
    tv.fit(sentences)

    words = list(tv.vocabulary_.keys())

    for word in words:
        df = sentiment_df[sentiment_df.ngram.str.contains(word, regex=False)]
        if len(df) == 0:
            continue
        else:
#             print(word)
            score = np.mean(-df.NEG + df.POS)
            scores.append(score)
            
    result = sum(scores)

    print("{} | {} | score = {}".format(date, title, result))

2017.03.10 | [사설] 헌재의 대통령 파면은 국민의 명령이다 | score = -6.150260271493018
2017.03.10 | [사설] 대선후보, 대통령 파면에서 무엇을 새겨야 하나 | score = 9.84734448507878
2017.03.10 | [사설] 오늘 승복이 법치의 새 역사 연다 | score = 9.456176038730044
2017.03.09 | [사설] 정치가 복원돼야 갈등 치유가 가능해진다 | score = -20.975039207110786
2017.03.08 | [사설] 3월 10일, 분열과 갈등의 끝이어야 한다 | score = -7.696425600127981
2017.03.06 | [사설] 특검은 재판, 검찰은 우병우 재수사에 매진하라 | score = -5.00807457059466
2017.03.05 | [사설] 운명의 한 주… 탄핵 이후를 준비할 때 | score = -6.873760086677365
2017.03.02 | [사설] 집회 전날 지지층 결집 부추긴 박 대통령 | score = 4.486922626314881
2017.03.02 | [사설] ‘대한독립 만세’ 98주년 … 갈라진 민심, 쪼개진 광장 | score = -12.566810399139992
2017.02.28 | [사설] 탄핵 찬반 선동이 3·1절 정신과 무슨 관계 있나 | score = 18.578356574042257
2017.02.27 | [사설] 역사적인 탄핵 심판정에 역사적 장면은 없었다 | score = -6.746764261774833
2017.02.27 | [사설] 특검의 수사 종결, 모두 ‘승복’하자 | score = -8.188294718913802
2017.02.26 | [사설] 갈등만 부추기는 대선주자들, 파국 원하나 | score = -8.39429591217066
2017.02.24 | [사설] 대선주자들이 먼저 헌재 결정 승복 밝혀라 | score = -9.557301951938697
2017.02.24 

### stop_words 만들기 (조사, 어미, 접두사, 접미사)

In [192]:
stop_words = []

for i in range(1220, 1221):
    date = ja_df.date[i]
    title = ja_df.title[i]
    print("{} | {}".format(date, title))
    
    sentences= kkma.sentences(ja_df.body[i])
    
    tv = TfidfVectorizer(ngram_range=(1, 1), tokenizer=tokenize_pos)
    tv.fit(sentences)

    words = list(tv.vocabulary_.keys())

    for word in words:
        if ("J" in word) or ("E" in word) or ("X" in word):
            stop_words.append(word)

2012.12.20 | [사설] 여성 대통령 박근혜 … 화려한 기록, 무거운 짐


In [196]:
stop_words

['가/JKS',
 '으로/JKM',
 '되/XSV',
 '었/EPT',
 '다/EFN',
 '은/JX',
 '도/JX',
 '화려/XR',
 '하/XSA',
 'ㄴ/ETD',
 '아/ECS',
 '은/ETD',
 '에서/JKM',
 '는/JX',
 '지만/ECE',
 '보다/JKM',
 '아/ECD',
 '있/VXV',
 '는/ETD',
 '이/JKS',
 '하/XSV',
 '은/JKS',
 '의/JKG',
 '에/JKM',
 '을/JKO',
 'ㄹ/ETD',
 '란/ETD',
 '나/JC',
 '와/JC',
 '를/JKO',
 '로/JKM',
 '았/EPT',
 '고/ECE',
 '으며/ECE',
 '이/JKC',
 '라는/ETD',
 '지/ECD',
 '않/VXV',
 '는/EPT',
 '아야/ECD',
 'ㄴ다/EFN',
 '과/JKM',
 'ㄴ데/ECE',
 '미약/XR',
 '위태/XR',
 '롭/XSA',
 '정/XSN',
 '면서/ECE',
 '며/ECE',
 '라도/JX',
 '면/ECE',
 '에게/JKM',
 '가/JKC',
 '과/JC',
 '어야/ECD',
 '어/ECS',
 '려는/ETD',
 'ㄴ단/ETD',
 '극심/XR',
 '처럼/JKM',
 '부터/JX',
 '을/ETD',
 '여차/XR',
 '려면/ECE',
 '와/JKM',
 '다/ECS',
 '과감/XR',
 '겠/EPT',
 '다는/ETD',
 '게/ECD',
 '뿐/JX',
 '라/ECD',
 'ㄴ가/EFQ',
 '막대/XR',
 '나/ECE',
 '성/XSN',
 '내/VXV',
 '들/XSN',
 '어서/ECD',
 '어/ECD']

In [195]:
len(list(set(stop_words)))

80

### 필요 단어 추가

In [5]:
ja_df.tail()

,date,title,body
1216,2012.12.24,"[사설] 박근혜·시진핑·아베, 과거 아닌 미래를 보라",역사는 사람이 만든다. 사람이 만드는 역사에서 리더의 역할은 결정적이다. 지금 동북...
1217,2012.12.22,[사설] 내년 예산안을 누더기로 만들 것인가,치열한 대선이 끝났다. 이제 정신을 차리고 발밑의 현실을 살펴야 할 때다. 국회가 ...
1218,2012.12.22,"[사설] 제2, 제3의 이학재 나와야 한다",박근혜 대통령 당선인의 핵심 측근인 이학재 새누리당 의원이 어제 백의종군을 선언했다...
1219,2012.12.21,[사설] 취임까지 67일이 5년 성패 좌우한다,박근혜 대통령 당선인이 어제 국민의 다양한 의견을 수렴하겠다고 약속했다. 또 “모든...
1220,2012.12.20,"[사설] 여성 대통령 박근혜 … 화려한 기록, 무거운 짐",박근혜 새누리당 후보가 18대 대통령으로 선출됐다. 여성 대통령은 한국은 물론 세계...


In [8]:
sentiment_df.tail()

,ngram,freq,COMP,NEG,NEUT,None,POS,max.value,max.prop
16357,힘들/VA;ㄹ/ETM;것/NNB,1,0.0,1.0,0.0,0.0,0.0,NEG,1.0
16358,힘들/VA;ㄹ/ETM;때/NNG,1,0.0,1.0,0.0,0.0,0.0,NEG,1.0
16359,힘차/VA,1,0.0,1.0,0.0,0.0,0.0,NEG,1.0
16360,힘차/VA;ㄴ/ETM,1,0.0,1.0,0.0,0.0,0.0,NEG,1.0
16361,힘차/VA;ㄴ/ETM;붓/NNG,1,0.0,1.0,0.0,0.0,0.0,NEG,1.0


In [16]:
keywords = []

for i in range(0, 1221):
    date = ja_df.date[i]
    title = ja_df.title[i]
    print("{} | {}".format(date, title))
    words = twitter.nouns(ja_df.body[i])
    
    for word in words:
        df = sentiment_df[sentiment_df.ngram.str.contains(word, regex=False)]
        if len(df) == 0:
            keywords += words

2017.03.10 | [사설] 헌재의 대통령 파면은 국민의 명령이다
2017.03.10 | [사설] 대선후보, 대통령 파면에서 무엇을 새겨야 하나
2017.03.10 | [사설] 오늘 승복이 법치의 새 역사 연다
2017.03.09 | [사설] 정치가 복원돼야 갈등 치유가 가능해진다
2017.03.08 | [사설] 3월 10일, 분열과 갈등의 끝이어야 한다
2017.03.06 | [사설] 특검은 재판, 검찰은 우병우 재수사에 매진하라
2017.03.05 | [사설] 운명의 한 주… 탄핵 이후를 준비할 때
2017.03.02 | [사설] 집회 전날 지지층 결집 부추긴 박 대통령
2017.03.02 | [사설] ‘대한독립 만세’ 98주년 … 갈라진 민심, 쪼개진 광장
2017.02.28 | [사설] 탄핵 찬반 선동이 3·1절 정신과 무슨 관계 있나
2017.02.27 | [사설] 역사적인 탄핵 심판정에 역사적 장면은 없었다
2017.02.27 | [사설] 특검의 수사 종결, 모두 ‘승복’하자
2017.02.26 | [사설] 갈등만 부추기는 대선주자들, 파국 원하나
2017.02.24 | [사설] 대선주자들이 먼저 헌재 결정 승복 밝혀라
2017.02.24 | [사설] 헌재 재판관 후임 지명, 아전인수식 해석 말라
2017.02.24 | [사설] 이정미 재판관 후임 지명, 빠를수록 좋다
2017.02.24 | [사설] 기소중지자가 될 박근혜 대통령
2017.02.23 | [사설] 두 보수당, 보수의 가치 알기는 하나
2017.02.22 | [사설] 탄핵시계 카운트다운 … 품격 있는 역사적 심판 돼야
2017.02.22 | [사설] 최순실의 권력기관장 꽂기 의혹 … 끝까지 진상 밝혀야
2017.02.22 | [사설] 교육부·미래부, 왜 해체 공약 나오는지 되새겨 보라
2017.02.21 | [사설] 박 대통령, 헌재 출석 일정 명백히 밝히는 게 도리다
2017.02.20 | [사설] 나라 찢어놓고 탄핵전쟁 승리하면 뭐하나
2017.02.17 | [사설] ‘영장기각 

In [17]:
len(keywords), len(list(set(keywords)))

(40777440, 14103)

In [10]:
keywords_sr = pd.DataFrame(data=keywords, columns=['Keywords']).Keywords.value_counts()

##### keyword for labeling

In [60]:
# 문제, 개혁, 책임, 의혹, 탄핵, 우려, 논란, 위기, 갈등, 비판, 지적, 사퇴, 혐의, 심판, 성과, 반대, 도발, 최고, 주의, 참사, 통합, 혼란, 평화, 비난, 농단, 퇴진, 거부, 이해, 참여, 반발, 압박, 의문, 인정, 위협, 낙하산, 사고, 중단, 촉구 투쟁, 분열, 비리, 협조, 지지, 기대, 가치, 수습, 쇄신, 회복, 실패, 한계, 잘못, 강화, 실천, 걱정, 분노, 포기, 효율, 긴장, 전쟁, 불법, 극복, 고민, 되풀이

In [13]:
list(keywords_sr.index)

['것',
 '대통령',
 '수',
 '정부',
 '박',
 '국민',
 '이',
 '등',
 '박근혜',
 '문제',
 '북한',
 '그',
 '청와대',
 '전',
 '대한',
 '국회',
 '게',
 '고',
 '정치',
 '말',
 '일',
 '우리',
 '개혁',
 '위',
 '인사',
 '대표',
 '안',
 '국정',
 '때',
 '경제',
 '새누리당',
 '의원',
 '수사',
 '검찰',
 '더',
 '국가',
 '중',
 '이번',
 '한국',
 '야당',
 '건',
 '때문',
 '상황',
 '장관',
 '대해',
 '중국',
 '명',
 '위해',
 '어제',
 '사실',
 '데',
 '점',
 '사회',
 '정책',
 '총리',
 '정권',
 '책임',
 '선',
 '여야',
 '대선',
 '지금',
 '기업',
 '사건',
 '당',
 '를',
 '일본',
 '의',
 '제',
 '의혹',
 '관련',
 '지난',
 '통해',
 '탄핵',
 '미국',
 '주장',
 '못',
 '관계',
 '두',
 '이유',
 '법',
 '대화',
 '복지',
 '민주당',
 '입장',
 '요구',
 '수석',
 '간',
 '회의',
 '또',
 '과정',
 '안보',
 '나라',
 '뒤',
 '세월호',
 '사람',
 '선거',
 '후보',
 '특검',
 '남북',
 '합의',
 '최',
 '집권',
 '결과',
 '모두',
 '연금',
 '외교',
 '규제',
 '새',
 '사태',
 '공무원',
 '세력',
 '협력',
 '조사',
 '공약',
 '정치권',
 '앞',
 '결정',
 '정',
 '일이',
 '가능성',
 '지난해',
 '대책',
 '비',
 '개',
 '우려',
 '위원회',
 '논란',
 '이후',
 '지역',
 '해결',
 '김',
 '핵심',
 '정도',
 '위기',
 '다른',
 '이상',
 '이제',
 '발표',
 '마련',
 '후보자',
 '실',
 '필요',
 '자신',
 '총선',
 '제대로',
 '경우',
 '갈

##### keypos

In [9]:
keypos = []

for i in range(0, 1221):
    date = ja_df.date[i]
    title = ja_df.title[i]
    print("{} | {}".format(date, title))
    pos = kkma.pos(ja_df.body[i])
    keypos += pos

2017.03.10 | [사설] 헌재의 대통령 파면은 국민의 명령이다
2017.03.10 | [사설] 대선후보, 대통령 파면에서 무엇을 새겨야 하나
2017.03.10 | [사설] 오늘 승복이 법치의 새 역사 연다
2017.03.09 | [사설] 정치가 복원돼야 갈등 치유가 가능해진다
2017.03.08 | [사설] 3월 10일, 분열과 갈등의 끝이어야 한다
2017.03.06 | [사설] 특검은 재판, 검찰은 우병우 재수사에 매진하라
2017.03.05 | [사설] 운명의 한 주… 탄핵 이후를 준비할 때
2017.03.02 | [사설] 집회 전날 지지층 결집 부추긴 박 대통령
2017.03.02 | [사설] ‘대한독립 만세’ 98주년 … 갈라진 민심, 쪼개진 광장
2017.02.28 | [사설] 탄핵 찬반 선동이 3·1절 정신과 무슨 관계 있나
2017.02.27 | [사설] 역사적인 탄핵 심판정에 역사적 장면은 없었다
2017.02.27 | [사설] 특검의 수사 종결, 모두 ‘승복’하자
2017.02.26 | [사설] 갈등만 부추기는 대선주자들, 파국 원하나
2017.02.24 | [사설] 대선주자들이 먼저 헌재 결정 승복 밝혀라
2017.02.24 | [사설] 헌재 재판관 후임 지명, 아전인수식 해석 말라
2017.02.24 | [사설] 이정미 재판관 후임 지명, 빠를수록 좋다
2017.02.24 | [사설] 기소중지자가 될 박근혜 대통령
2017.02.23 | [사설] 두 보수당, 보수의 가치 알기는 하나
2017.02.22 | [사설] 탄핵시계 카운트다운 … 품격 있는 역사적 심판 돼야
2017.02.22 | [사설] 최순실의 권력기관장 꽂기 의혹 … 끝까지 진상 밝혀야
2017.02.22 | [사설] 교육부·미래부, 왜 해체 공약 나오는지 되새겨 보라
2017.02.21 | [사설] 박 대통령, 헌재 출석 일정 명백히 밝히는 게 도리다
2017.02.20 | [사설] 나라 찢어놓고 탄핵전쟁 승리하면 뭐하나
2017.02.17 | [사설] ‘영장기각 

In [10]:
len(keypos), len(list(set(keypos)))

(835036, 19926)

In [25]:
for i in range(len(keypos)):
    keypos[i] = "/".join(keypos[i])

In [28]:
pd.DataFrame(data=keypos, columns=['keypos']).keypos.value_counts()

./SF             30007
다/EFN            23931
하/XSV            23510
ㄴ/ETD            22668
을/JKO            19076
의/JKG            16423
이/VCP            15473
이/JKS            14562
에/JKM            12823
를/JKO            11244
었/EPT            10950
는/ETD            10739
은/JX              9771
고/ECE             8694
하/VV              8396
ㄹ/ETD             8212
는/JX              8033
어/ECS             7840
가/JKS             7734
것/NNB             7001
도/JX              5817
대통령/NNG           5412
으로/JKM            5289
ㄴ다/EFN            5126
에서/JKM            4817
·/SP              4647
,/SP              4557
되/XSV             4288
있/VV              3923
로/JKM             3654
                 ...  
앗/VV                 1
포커/NNG               1
舌禍/OH                1
허영/NNG               1
TI/OL                1
금형/NNG               1
성탄절/NNG              1
1444/NR              1
3800/NR              1
술렁/XR                1
434/NR               1
익히/VV                1
상통/NNG     